In [ ]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof

from matplotlib import pyplot as plt

In [ ]:
#From signac
import signac
date_time_str = None
meth_name_str_list = [1, 2, 3, 4, 5]
study_id = "ep"
save_csv = False
param_name_str = "t1t2"
cs_name_val = 1
project = signac.get_project()
df = pd.DataFrame()
for meth_name_val in meth_name_str_list:
    df_piece, name_cs_str, theta_true = get_study_data_signac(project, cs_name_val, param_name_str, meth_name_val, 
                                                              study_id, save_csv)
    df = pd.concat([df, df_piece], ignore_index=True)
    
# df = df[df['EP Method Val'] == 4]
# df

In [ ]:
#Get Best Data from ep experiment
save_csv = False
df_best = get_best_data(df, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)
df_best

# #Get Mean Data from ep experiment
# df_mean = get_mean_data(df, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)
# df_mean

# #Get Median Data from ep experiment
# df_median = get_median_data(df, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)

# #Get Median of Best Data from ep experiment
# df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)
# df_median_best

In [ ]:
#From signac
import signac
date_time_str = None
meth_name_str_list = [1, 2, 3, 5]
study_id = "sf"
save_csv = True
cs_name_val = 1
param_name_str = "t1t2"
project = signac.get_project()
df = pd.DataFrame()
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# name_cs_str = "CS1" 
# meth_name_str_list = ["A1", "B1", "A2"]

#Get all sf data signac
df = pd.DataFrame()
for meth_name_val in meth_name_str_list:
    df_piece, name_cs_str, theta_true = get_study_data_signac(project, cs_name_val, param_name_str, meth_name_val, study_id, save_csv)
    df = pd.concat([df, df_piece], ignore_index=True)
    
df = df.sort_values(by=['BO Method','Sep Fact'], ascending=True)

In [ ]:
file_path_list = []
project = signac.get_project()
param_name_str = "t1t2"
cs_name_val = 1
meth_name_str_list = [1, 2, 3, 5]

for meth_name_val in meth_name_str_list:
    meth_name = Method_name_enum(meth_name_val)
    cs_name_enum = CS_name_enum(cs_name_val) 
    #Get best ep data from Results path if possible
    path_name = "Results/ep_study/" + cs_name_enum.name + "/" + param_name_str + "/ep_study_best.csv"
    if os.path.exists(path_name):
        df_ep_best = pd.read_csv(path_name, header = 0, index_col = 0)
    #If there is no results path infer it directly from the jobs
    else:
        df_ep, cs_name, theta_true = get_study_data_signac(project, cs_name_val, param_name_str, meth_name_val,
                                                           "ep", save_csv = False) 
        df_ep_best = get_best_data(df_ep, "ep", cs_name, theta_true, param_name_str, date_time_str = None, 
                                   save_csv = False)

    #Set ep enum val to the best one for that cs and method
    best_ep_enum_val = int(df_best.loc[df_best['BO Method'] == meth_name.name, 'EP Method Val'].iloc[0])

    #Get all jobs with that ep enum val
    jobs = project.find_jobs({"cs_name_val": cs_name_val, "meth_name_val" : meth_name_val, "ep_enum_val": best_ep_enum_val})
    for job in jobs:
        file_path_list.append(job.fn("BO_Results.gz"))

In [ ]:
#Get Best Data from sf experiment
df_best = get_best_data(df, study_id, name_cs_str, theta_true, param_name_str, date_time_str, save_csv)
df_best

# #Get Median Data from sf experiment
# df_median = get_median_data(df, study_id, name_cs_str, theta_true, param_name_str,  date_time_str, save_csv)

# #Get Median of Best Data from sf experiment
# df_median_best = get_median_data(df_best, study_id, name_cs_str, theta_true, param_name_str,  date_time_str, save_csv)
# df_median_best

In [ ]:
title_fontsize = 24
other_fontsize = 20
xbins = 5
ybins = 7
save_path = None
data_true = None
log_data = False
title = None

plot_compare_mean_med_best(df, name_cs_str, theta_true, xbins, ybins, title, log_data, title_fontsize, 
                           other_fontsize, save_path)

In [ ]:
#Get separation factor experiment graphs

title_fontsize = 24
other_fontsize = 20
xbins = 5
ybins = 7
x_label = "Separation Factor"
y_label = "SSE"
save_path = None
data_true = None
log_data = False
title = "SF Analysis"

f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
precision = 1
sep_fact_list = [f"{number:.{precision}f}" for number in f] #Use f for signac and sep_fact_list for original

plot_all_SF_data(df, xbins, ybins, title, x_label, y_label, log_data, title_fontsize, other_fontsize, save_path)